In [1]:
# Comments

# [cell]

# imports

from __future__ import absolute_import, division, print_function

import ipywidgets
import radia

from jupyter_rs_radia import radia_viewer

# [cell]

# vars

# radia viewer
rv = radia_viewer.RadiaViewer()

# Geometry Parameters
gap = 10 #(mm)
thick = 50
width = 40
chamfer = 8 #(mm)
current = -2000 #(A)

# Segmentation Parameters
nx = 2
nbp = 2
nbr = 2 #for corners
n1 = [nx,3,2] #pole faces
n2 = [nx,2,2] #small vertical arm
n3 = [nx,2,2]
n4 = [nx,2,2] #horizontal arm
n5 = [nx,2,2]
n6 = [nx,2,2] #inside the coil

# [cell]
# functions and handlers

def Geom(circ):

    eps = 0
    ironcolor = [0, 0.5, 1]
    coilcolor = [1, 0, 0]
    ironmat = radia.MatSatIsoFrm([20000, 2], [0.1, 2], [0.1, 2])
    
    # Pole faces
    lx1 = thick/2
    ly1 = width
    lz1 = 20
    l1 = [lx1, ly1, lz1]
    
    k1 = [[thick/4. - chamfer/2., 0, gap/2.], [thick/2. - chamfer, ly1 - 2.*chamfer]]
    k2 = [[thick/4., 0., gap/2. + chamfer], [thick/2., ly1]]
    k3 = [[thick/4., 0. ,gap/2. + lz1], [thick/2, ly1]]
    g1 = radia.ObjMltExtRtg([k1, k2, k3])
    radia.ObjDivMag(g1, n1)
    radia.ObjDrwAtr(g1, ironcolor)
    rv.add_geom('Pole', g1)

    #Vertical segment on top of pole faces
    lx2 = thick/2
    ly2 = ly1
    lz2 = 30
    l2 = [lx2, ly2, lz2]
    p2 = [thick/4, 0, lz1 + gap/2 + lz2/2 + 1*eps]
    g2 = radia.ObjRecMag(p2, l2)
    radia.ObjDivMag(g2, n2)
    radia.ObjDrwAtr(g2, ironcolor)
    rv.add_geom('Vertical Pole Segment', g2)

    #Corner
    lx3 = thick/2
    ly3 = ly2
    lz3 = ly2*1.25
    l3 = [lx3, ly3, lz3]
    p3 = [thick/4, 0, lz1 + gap/2 + lz2 + lz3/2 + 2*eps]
    g3 = radia.ObjRecMag(p3, l3)

    typ = [
        [p3[0], p3[1] + ly3/2, p3[2] - lz3/2],
        [1, 0, 0],
        [p3[0], p3[1]-ly3/2, p3[2]-lz3/2],
        lz3/ly3
    ]

    if circ == 1:
        radia.ObjDivMag(g3, [nbr, nbp, n3[1]], 'cyl', typ)
    else:
        radia.ObjDivMag(g3, n3)        
    radia.ObjDrwAtr(g3, ironcolor)
    rv.add_geom('Corner 1', g3)

    # Horizontal segment between the corners
    lx4 = thick/2
    ly4 = 80
    lz4 = lz3
    l4 = [lx4, ly4, lz4]
    p4 = [thick/4, ly3/2 + eps + ly4/2, p3[2]]
    g4 = radia.ObjRecMag(p4, l4)
    radia.ObjDivMag(g4, n4)
    radia.ObjDrwAtr(g4, ironcolor)
    rv.add_geom('Horizontal Corner Segment', g4)

    #The other corner
    lx5 = thick/2
    ly5 = lz4*1.25
    lz5 = lz4
    l5 = [lx5, ly5, lz5]
    p5 = [thick/4, p4[1] + eps + (ly4 + ly5)/2, p4[2]]
    g5 = radia.ObjRecMag(p5, l5)

    typ = [
        [p5[0], p5[1] - ly5/2, p5[2] - lz5/2],
        [1, 0, 0],
        [p5[0], p5[1] + ly5/2, p5[2] - lz5/2],
        lz5/ly5
    ]

    if circ == 1:
        radia.ObjDivMag(g5, [nbr, nbp, n5[0]], 'cyl', typ)
    else:
        radia.ObjDivMag(g5, n5)
    radia.ObjDrwAtr(g5, ironcolor)
    rv.add_geom('Corner 2', g5)

    # Vertical segment inside the coil
    lx6 = thick/2
    ly6 = ly5
    lz6 = gap/2 + lz1 + lz2
    l6 = [lx6, ly6, lz6]
    p6 = [thick/4, p5[1], p5[2] - (lz6 + lz5)/2 - eps]
    g6 = radia.ObjRecMag(p6, l6)
    radia.ObjDivMag(g6, n6)
    radia.ObjDrwAtr(g6, ironcolor)
    rv.add_geom('Vertical Coil Segment', g6)

    #Generation of the coil
    Rmin = 5
    Rmax = 40
    H = 2 * lz6 - 5
    
    CurDens = current / H / (Rmax - Rmin)
    pc = [0, p6[1], 0]
    coil = radia.ObjRaceTrk(pc, [Rmin, Rmax], [thick, ly6], H, 3, CurDens)
    radia.ObjDrwAtr(coil, coilcolor)
    rv.add_geom('Coil', coil)

    #Make container and set the colors
    g = radia.ObjCnt([g1, g2, g3, g4, g5, g6])
    g_base = radia.ObjCnt([g1, g2, g3, g4, g5, g6])
    rv.add_geom('Iron Base', g_base)
    rv.add_geom('Iron', g)
    radia.ObjDrwAtr(g, ironcolor)
    radia.MatApl(g, ironmat)
    t = radia.ObjCnt([g, coil])

    #Define the symmetries
    radia.TrfZerPerp(g, [0, 0, 0], [1, 0, 0])
    radia.TrfZerPara(g, [0, 0, 0], [0, 0, 1])
    return t


# [cell]
# define full magnet
magnet = Geom(1)
rv.add_geom('Magnet', magnet)


AttributeError: module 'jupyter_rs_vtk.rs_utils' has no attribute 'to_pkdict'

In [ ]:
# show radia viewer
rv.display()
